<p float="left">
  <img src=nanoHUB_logo_color.png width="500px" height='200px' align="center" /> 
</p>


# Querying and Managing Data from Materials Project


<b> nanoHUB tools by: </b>  <i>Zachary D. McClure</i> and <i>Alejandro Strachan</i>, Materials Engineering, Purdue University <br>
### Authorship and credits

<b> Based on work published by: </b> [Shyue Ping Ong et al.](https://www.sciencedirect.com/science/article/abs/pii/S0927025612006295?via%3Dihub) <br> 


## Overview

This notebook will serve as a workflow for querying a large set of data from the Materials Project database. As an example we will query a large set of oxide compounds from the database and attempt to filter out unnecessary results. Our primary tools will be the Materials Project API and the Pandas dataframe library.

**Outline:**

1. Querying an oxide dataset Materials Project <br>
2. Filtering data for convex hull energy<br>
3. Filtering data for elastic constants <br>

Notes: 
* This notebook uses tools from [Materials Project](https://materialsproject.org/open) and requires an account with an API key.
* Run cells in sequential order. In many cases we will drop columns or rows from dataframes, and re-running the cell will error out. To re-run notebook begin with cells below the initial query.

# Loading your API Key
In order to load your API key, you can insert it in the Input textbox below and we will store it for your convenience. Remember to hit "Enter". 

### Load Materials Project API Key

In [1]:
import os, stat
from IPython.display import clear_output

try:
    user = str(input())
    clear_output()
    if not user.isalnum():
        raise TypeError('Wrong Key')
    if user == None:
        raise TypeError('Empty')
    with open(os.path.expanduser('~/.mpkey.txt'), 'w') as keyfile:
        keyfile.write(user)
    os.chmod(os.path.expanduser('~/.mpkey.txt'), stat.S_IREAD | stat.S_IWRITE)
    del user
    print("Success")
except:
    print("Something seems wrong with your key")

Success


## Libraries

This notebook requires several libraries to be imported. They are separated in blocks depending on their usage.

In [2]:
# These lines import both libraries and then define an array with elements to be used below

#Pymatgen and sub-libraries
import pymatgen
from pymatgen import MPRester, Element, Structure
from pymatgen.core import composition
from pymatgen.io.vasp import Vasprun
from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.entries.compatibility import MaterialsProjectCompatibility
from pymatgen.util.plotting import *
from pymatgen.analysis.phase_diagram import *
#import pymatgen.core.composition
from pymatgen.apps.borg.hive import VaspToComputedEntryDrone
from pymatgen.entries.compatibility import MaterialsProjectCompatibility
import json

#Misc. Tools
import matplotlib.pyplot as plt
import random
import os
from itertools import combinations
import pandas as pd

#Import API Key
file = open(os.path.expanduser('~/.mpkey.txt'),"r+")
apikey = file.readline()
file.close()
rester = MPRester(apikey)

## 1. Query all oxides in MP database

In [3]:
#Query every structure in the MP database that has oxygen atom within compound. 
#Grab set of properties from database. Ti and Fe based.
data = rester.query({"elements": "Ti", "nelements": {"$gte": 2}},
                    ["task_id","pretty_formula","formula","volume","density","elements",
                     "e_above_hull","elasticity","unit_cell_formula"])

First let's check how large our queried database is. Print out the length of the entire database the find the total number of oxide compounds that exist of the MP database

In [4]:
print('Number of oxide structures available on the MP database: %s' % len(data))
print('Example output: %s' % data[24])

Number of oxide structures available on the MP database: 5846
Example output: {'task_id': 'mp-1062030', 'pretty_formula': 'TiS2', 'formula': {'Ti': 1.0, 'S': 2.0}, 'volume': 114.22459036770374, 'density': 1.6281554634094206, 'elements': ['S', 'Ti'], 'e_above_hull': 0.0035409266666670547, 'elasticity': None, 'unit_cell_formula': {'Ti': 1.0, 'S': 2.0}}


## 2. Filtering data for energy stability
After data query has been completed we will begin filtering things just a bit. First we will strip any values that have an energy above the convex hull greater than 1 meV. 0 meV indicates most stable phase. This filter can be changed below.<br>

In the same loop we will calculate the ionic packing fraction of each oxide. We start by querying atomic radii for each element, and performing a quick calculation as shown above. For initial speed we keep everything as easily accessible dictionaries. In the following cells we will convert to a Pandas dataframe which is easier for filtering and plotting data.

In [5]:
energy_cutoff_value = 1 #energy above convex hull cutoff [meV]

### 2.1 Filter data and calculate ionic packing fraction from compound query

In [6]:
#Create temporary list arrays for data calculation and manipulation. We will clear the array after each loop
atom_sample = []
atom_count = []
atom_radii = []
atom_volume = []
atom_volume_total = []

#Create modified dictionary to store values that we want from queried database of all oxides
property_dict_all = {}

for k in range(0,len(data)):
    #If Energy above convex Hull is larger than 1 meV we will skip that value and move on
    e_above_hull = data[k].get('e_above_hull')*1000
    if e_above_hull > energy_cutoff_value:
        continue
    #Grab values from data query. Data stored as dictionary so we call the 'keys' for 'values'
    mp_id = data[k].get('task_id')
    density = data[k].get('density')
    pretty_formula = data[k].get('pretty_formula')
    elastic_tensor = data[k].get('elasticity')
    composition = data[k].get('formula')
    oxide_volume = data[k].get('volume')
    atom_type_count = len(data[k].get('elements'))
    unit_cell_formula = data[k].get('unit_cell_formula')

    #Pull atom count for each atom type. Will be used to find summed atomic volumes and summed masses
    for type in np.arange(0,atom_type_count):
        atom_sample.append(data[k].get('elements')[type])     
        atom_count.append(data[k].get('unit_cell_formula').get(data[k].get('elements')[type]))
        
    #Do a quick query of each element type for ionic radius and atomic mass from Pymatgen
    for item in atom_sample:
        element_object = pymatgen.Element(item)
        radii_values = element_object.average_ionic_radius
        atom_radii.append(radii_values)

    #For each atom radii queried calculate the atomic volume
    for i in atom_radii:
        v = (4/3)*np.pi*i**3
        atom_volume.append(v)
        
    #Calculate the total atomic volume for each element type (E.g. 3*(Volume O atom))
    for i in np.arange(0,len(atom_count)):
        v_total = atom_count[i]*atom_volume[i]
        atom_volume_total.append(v_total)
        
    #Sum the volumes of all atoms in the oxide
    summed_volumes = sum(atom_volume_total)
    
    #Calculate packing fraction. Append values to dictionary with the Materials Project ID as key value
    packing_frac = summed_volumes/oxide_volume
    property_dict_all[str(mp_id)] = pretty_formula,composition,elastic_tensor,packing_frac,density,e_above_hull
    
    #Clear values for next loop
    atom_sample.clear()
    atom_count.clear()
    atom_radii.clear()
    atom_volume.clear()
    atom_volume_total.clear()

### 2.2 Convert our dictionary to a more pleasant looking dataframe. Pull the values from each key, separate them, and shove everything into a dataframe.

In [7]:
#Create blank lists for appending values
mp_id = []
pretty_formula = []
composition = []
ionic_packing_frac = []
density = []
elastic_tensor = []
e_above_hull = []
#Loop through all keys and extract their values. Append values into individual list arrays
for key in property_dict_all.keys():
    mp_id.append(key)
    values = property_dict_all.get(key)
    pretty_formula.append(values[0])
    composition.append(values[1])
    elastic_tensor.append(values[2])
    ionic_packing_frac.append(values[3])
    density.append(values[4])
    e_above_hull.append(values[5])    
#Stack arrays into a Numpy array. From here will append into a dataframe. Display dataframe below.
df = np.column_stack((mp_id,pretty_formula,composition,ionic_packing_frac,density,elastic_tensor,e_above_hull))
df_oxide = pd.DataFrame(df,columns=["mp_id","Formula","Composition","IPF","Density","Elastic_Tensor","E_above_Hull"])
display(df_oxide)

,mp_id,Formula,Composition,IPF,Density,Elastic_Tensor,E_above_Hull
0,mp-1014229,Ti2Zn,"{'Ti': 2.0, 'Zn': 1.0}",0.163919,5.46244,"{'G_Reuss': 29.0, 'G_VRH': 47.0, 'G_Voigt': 65...",0
1,mp-1014230,TiZn,"{'Ti': 1.0, 'Zn': 1.0}",0.174651,6.03654,"{'G_Reuss': 44.0, 'G_VRH': 56.0, 'G_Voigt': 69...",0
2,mp-1017983,TiMo3,"{'Ti': 1.0, 'Mo': 3.0}",0.133018,8.78825,None,0
3,mp-1017985,TiAg,"{'Ti': 1.0, 'Ag': 1.0}",0.226014,7.34036,"{'G_Reuss': 32.0, 'G_VRH': 35.0, 'G_Voigt': 37...",0
4,mp-1018028,TiS,"{'Ti': 1.0, 'S': 1.0}",0.18213,4.44203,"{'G_Reuss': 97.0, 'G_VRH': 101.0, 'G_Voigt': 1...",0
...,...,...,...,...,...,...,...
1038,mp-696940,TiH12C2(NF)6,"{'Ti': 1.0, 'H': 12.0, 'C': 2.0, 'N': 6.0, 'F'...",0.0644307,1.68517,None,0
1039,mp-720394,Ca5TaTi3Al(SiO5)5,"{'Ca': 5.0, 'Ta': 1.0, 'Ti': 3.0, 'Al': 1.0, '...",0.536486,3.81363,None,0
1040,mp-720410,K2NaCa2TiSi7HO20,"{'K': 2.0, 'Na': 1.0, 'Ca': 2.0, 'Ti': 1.0, 'S...",0.48721,2.70744,None,0
1041,mp-744094,Ba2La6Mg4Ti3WO24,"{'Ba': 2.0, 'La': 6.0, 'Mg': 4.0, 'Ti': 3.0, '...",0.517511,5.68804,None,0


In [8]:
df_oxide.dtypes

mp_id             object
Formula           object
Composition       object
IPF               object
Density           object
Elastic_Tensor    object
E_above_Hull      object
dtype: object

In [9]:
df_oxide['Elastic_Tensor']

0       {'G_Reuss': 29.0, 'G_VRH': 47.0, 'G_Voigt': 65...
1       {'G_Reuss': 44.0, 'G_VRH': 56.0, 'G_Voigt': 69...
2                                                    None
3       {'G_Reuss': 32.0, 'G_VRH': 35.0, 'G_Voigt': 37...
4       {'G_Reuss': 97.0, 'G_VRH': 101.0, 'G_Voigt': 1...
                              ...                        
1038                                                 None
1039                                                 None
1040                                                 None
1041                                                 None
1042                                                 None
Name: Elastic_Tensor, Length: 1043, dtype: object

In [10]:
df_oxide=df_oxide.dropna()

In [11]:
df_oxide

,mp_id,Formula,Composition,IPF,Density,Elastic_Tensor,E_above_Hull
0,mp-1014229,Ti2Zn,"{'Ti': 2.0, 'Zn': 1.0}",0.163919,5.46244,"{'G_Reuss': 29.0, 'G_VRH': 47.0, 'G_Voigt': 65...",0
1,mp-1014230,TiZn,"{'Ti': 1.0, 'Zn': 1.0}",0.174651,6.03654,"{'G_Reuss': 44.0, 'G_VRH': 56.0, 'G_Voigt': 69...",0
3,mp-1017985,TiAg,"{'Ti': 1.0, 'Ag': 1.0}",0.226014,7.34036,"{'G_Reuss': 32.0, 'G_VRH': 35.0, 'G_Voigt': 37...",0
4,mp-1018028,TiS,"{'Ti': 1.0, 'S': 1.0}",0.18213,4.44203,"{'G_Reuss': 97.0, 'G_VRH': 101.0, 'G_Voigt': 1...",0
5,mp-1018123,TiAu2,"{'Ti': 1.0, 'Au': 2.0}",0.248079,14.1627,"{'G_Reuss': -92.0, 'G_VRH': -17.0, 'G_Voigt': ...",0
...,...,...,...,...,...,...,...
907,mp-5999,Sm2Ti2S2O5,"{'Sm': 2.0, 'Ti': 2.0, 'S': 2.0, 'O': 5.0}",0.398389,5.23354,"{'G_Reuss': 54.0, 'G_VRH': 58.0, 'G_Voigt': 62...",0
909,mp-6328,Na2Ti2Sb2O,"{'Na': 2.0, 'Ti': 2.0, 'Sb': 2.0, 'O': 1.0}",0.217025,4.60188,"{'G_Reuss': 27.0, 'G_VRH': 18.0, 'G_Voigt': 10...",0
920,mp-998980,TiAlFeCo,"{'Ti': 1.0, 'Al': 1.0, 'Fe': 1.0, 'Co': 1.0}",0.170756,6.42314,"{'G_Reuss': 127.0, 'G_VRH': 127.0, 'G_Voigt': ...",0
966,mp-560767,KNaTiO3,"{'K': 1.0, 'Na': 1.0, 'Ti': 1.0, 'O': 3.0}",0.551255,2.95236,"{'G_Reuss': 18.0, 'G_VRH': 26.0, 'G_Voigt': 33...",0


In [17]:
df_oxide.to_csv (r'C:\Users\Kayla Yano\Class\Trial\Ti_binary_dataframe.csv', index = True, header=True)

### 2.3 Observe the reduction from all oxide species to the limited set for stable structures. Plot the data below.

In [12]:
print('Number of oxide structures available on the MP database: %s' % len(data))
print('Number of oxide structures below %.1f meV: %s' %(energy_cutoff_value, len(df_oxide)))

Number of oxide structures available on the MP database: 5846
Number of oxide structures below 1.0 meV: 338


In [13]:
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot

plotly.offline.init_notebook_mode(connected=True)

#Layout design. Can change title, font, x/y-axis etc. Commented out pieces are height and width. 
#For plot in browser I prefer long horizontal plots. For presentations square images may be preferred. 
#Image can be directly saved by hovering over image and clicking camera icon. 
layout_IPF = go.Layout(title= "Oxide Query IPF vs. Density", hovermode= 'closest',
                   font = dict(family='Times New Roman',size=18),
                   xaxis= dict(title= '$Density \: [g/cm^{3}]$',zeroline= False, gridwidth= 2),
                   yaxis= dict(title= '$Ionic \: Packing \: Fraction \: [IPF]$',zeroline= False, gridwidth= 2),
                   height = 600,
                   width = 1200,
     showlegend= True                
)

#Scatter plot of collected data. Use df_oxide_all dataframe. df_oxide_all.Density will pull density column.
trace_all = go.Scatter(x = df_oxide.Density, y = df_oxide.IPF, mode = 'markers',
                    marker=dict(size=12, color='black'), text = df_oxide.Formula, name = 'All Queries')

trace_Al2O3 = go.Scatter(x = df_oxide[(df_oxide.Formula == 'Al2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                         y = df_oxide[(df_oxide.Formula == 'Al2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                         mode = 'markers', marker=dict(size=18, color='green'), text = 'Al2O3', name = 'Al2O3')
trace_Cr2O3 = go.Scatter(x = df_oxide[(df_oxide.Formula == 'Cr2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                         y = df_oxide[(df_oxide.Formula == 'Cr2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                         mode = 'markers', marker=dict(size=18, color='blue'), text = 'Cr2O3', name = 'Cr2O3')
trace_Y2O3 = go.Scatter(x = df_oxide[(df_oxide.Formula == 'Y2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                        y = df_oxide[(df_oxide.Formula == 'Y2O3') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                        mode = 'markers', marker=dict(size=18, color='orange'), text = 'Y2O3', name = 'Y2O3')
trace_SiO2 = go.Scatter(x = df_oxide[(df_oxide.Formula == 'SiO2') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                        y = df_oxide[(df_oxide.Formula == 'SiO2') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                        mode = 'markers', marker=dict(size=18, color='magenta'), text = 'SiO2', name = 'SiO2')

data_IPF = [trace_all,trace_Al2O3,trace_Cr2O3,trace_Y2O3,trace_SiO2]#,trace0,trace2]
fig_IPF = go.Figure(data_IPF, layout=layout_IPF)
iplot(fig_IPF)

## 3. Filtering data for elasticity

In [14]:
void_test = []
row_delete = []
for k in range(0,len(df_oxide)):
    void_test.append(str(df_oxide.loc[k,'Elastic_Tensor']))
    if void_test.count('None') == 1:
        void_test.clear()
        row_delete.append(k)
    if void_test.count('nan') == 1:
        void_test.clear()
        row_delete.append(k)

index = df_oxide.index[row_delete]
df_oxide.drop(index,axis=0,inplace=True)
df_elasticity_filtered = df_oxide
display(df_elasticity_filtered)

KeyError: 2

### 3.1 Observe the reduction from all oxide species to the limited set for stable structures, and ones with mechanical property data


In [15]:
print('Number of oxide structures available on the MP database: %s' % len(data))
print('Number of oxide structures below %.1f meV and with elasticity data: %s' %(energy_cutoff_value, len(df_oxide)))

Number of oxide structures available on the MP database: 5846
Number of oxide structures below 1.0 meV and with elasticity data: 338


### 3.2 Pull dataframe column with dictionary of elastic constants. Create separate columns for each value instead of inset in dictionary.

In [16]:
#Pull dataframe column with dictionary of elastic constants
elastics = df_oxide.Elastic_Tensor
G_VRH = []
K_VRH = []
Elastic_Anisotropy = []
poisson_ratio = []
Y_Modulus = []

for k in elastics.index: #len(df_oxide))
    G_VRH.append(elastics[k].get('G_VRH'))
    K_VRH.append(elastics[k].get('K_VRH'))

    Elastic_Anisotropy.append(elastics[k].get('elastic_anisotropy'))
    poisson_ratio.append(elastics[k].get('poisson_ratio'))
    Y_Modulus.append(2*elastics[k].get('G_VRH')*(1+elastics[k].get('poisson_ratio')))
    
elasticity_data = np.column_stack((G_VRH, K_VRH, Elastic_Anisotropy, poisson_ratio, Y_Modulus))

df_elasticity = pd.DataFrame(elasticity_data,columns=["G_VRH", 
                                                      "K_VRH", 
                                                      "Elastic_Anisotropy", 
                                                      "poisson_ratio",
                                                     "Y_Modulus"])



df_oxide = df_oxide.reset_index(drop=True)
df_elasticity = df_elasticity.reset_index(drop=True)
df_oxide = pd.concat([df_oxide,df_elasticity],axis=1)
display(df_oxide)

#Remove erroneous poisson ratios
void_test = []
row_delete = []
for k in range(0,len(df_elasticity)):
    if df_oxide['poisson_ratio'].iloc[k] < 0:
        row_delete.append(k)
        

index = df_oxide.index[row_delete]
df_oxide.drop(index,axis=0,inplace=True)

,mp_id,Formula,Composition,IPF,Density,Elastic_Tensor,E_above_Hull,G_VRH,K_VRH,Elastic_Anisotropy,poisson_ratio,Y_Modulus
0,mp-1014229,Ti2Zn,"{'Ti': 2.0, 'Zn': 1.0}",0.163919,5.46244,"{'G_Reuss': 29.0, 'G_VRH': 47.0, 'G_Voigt': 65...",0,47.0,96.0,6.32,0.29,121.26
1,mp-1014230,TiZn,"{'Ti': 1.0, 'Zn': 1.0}",0.174651,6.03654,"{'G_Reuss': 44.0, 'G_VRH': 56.0, 'G_Voigt': 69...",0,56.0,115.0,2.85,0.29,144.48
2,mp-1017985,TiAg,"{'Ti': 1.0, 'Ag': 1.0}",0.226014,7.34036,"{'G_Reuss': 32.0, 'G_VRH': 35.0, 'G_Voigt': 37...",0,35.0,111.0,0.80,0.36,95.20
3,mp-1018028,TiS,"{'Ti': 1.0, 'S': 1.0}",0.18213,4.44203,"{'G_Reuss': 97.0, 'G_VRH': 101.0, 'G_Voigt': 1...",0,101.0,141.0,0.44,0.21,244.42
4,mp-1018123,TiAu2,"{'Ti': 1.0, 'Au': 2.0}",0.248079,14.1627,"{'G_Reuss': -92.0, 'G_VRH': -17.0, 'G_Voigt': ...",0,-17.0,130.0,-8.12,0.57,-53.38
...,...,...,...,...,...,...,...,...,...,...,...,...
333,mp-5999,Sm2Ti2S2O5,"{'Sm': 2.0, 'Ti': 2.0, 'S': 2.0, 'O': 5.0}",0.398389,5.23354,"{'G_Reuss': 54.0, 'G_VRH': 58.0, 'G_Voigt': 62...",0,58.0,134.0,0.77,0.31,151.96
334,mp-6328,Na2Ti2Sb2O,"{'Na': 2.0, 'Ti': 2.0, 'Sb': 2.0, 'O': 1.0}",0.217025,4.60188,"{'G_Reuss': 27.0, 'G_VRH': 18.0, 'G_Voigt': 10...",0,18.0,52.0,-2.81,0.34,48.24
335,mp-998980,TiAlFeCo,"{'Ti': 1.0, 'Al': 1.0, 'Fe': 1.0, 'Co': 1.0}",0.170756,6.42314,"{'G_Reuss': 127.0, 'G_VRH': 127.0, 'G_Voigt': ...",0,127.0,175.0,0.01,0.21,307.34
336,mp-560767,KNaTiO3,"{'K': 1.0, 'Na': 1.0, 'Ti': 1.0, 'O': 3.0}",0.551255,2.95236,"{'G_Reuss': 18.0, 'G_VRH': 26.0, 'G_Voigt': 33...",0,26.0,56.0,4.12,0.30,67.60


### 3.3.1 Plot material properties from dataframe

In [21]:
#Layout design. Can change title, font, x/y-axis etc. Commented out pieces are height and width. For plot in browser I prefer long horizontal plots. For presentations square images may be preferred. 
#Image can be directly saved by hovering over image and clicking camera icon. 
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot

layout_IPF = go.Layout(title= "Ti_based Query IPF vs. Density", hovermode= 'closest',
                   font = dict(family='Times New Roman',size=18),
                   xaxis= dict(title= '$Density \: [g/cm^{3}]$',zeroline= False, gridwidth= 2),
                   yaxis= dict(title= '$Ionic Packing Fraction \: [IPF]$',zeroline= False, gridwidth= 2),
                   height = 600,
                   width = 1200,
     showlegend= True                
)

#Scatter plot of collected data. Use df_oxide_all dataframe. df_oxide_all.Density will pull density column. Could change Density or IPF to Molar Volume or simply volume if you wanted.
trace_all = go.Scatter(x = df_oxide.Density, y = df_oxide.IPF, mode = 'markers',
                    marker=dict(size=12, color='black'), text = df_oxide.Formula, name = 'All Queries')

trace_TiC = go.Scatter(x = df_oxide[(df_oxide.Formula == 'TiC') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                         y = df_oxide[(df_oxide.Formula == 'TiC') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                         mode = 'markers', marker=dict(size=18, color='green'), text = 'TiC', name = 'TiC')
trace_TiN = go.Scatter(x = df_oxide[(df_oxide.Formula == 'TiN') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                         y = df_oxide[(df_oxide.Formula == 'TiN') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                         mode = 'markers', marker=dict(size=18, color='blue'), text = 'TiN', name = 'TiN')
trace_TiB = go.Scatter(x = df_oxide[(df_oxide.Formula == 'TiB') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                        y = df_oxide[(df_oxide.Formula == 'TiB') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                        mode = 'markers', marker=dict(size=18, color='orange'), text = 'TiB', name = 'TiB')
trace_TiO2 = go.Scatter(x = df_oxide[(df_oxide.Formula == 'TiO2') & (df_oxide.E_above_Hull == 0)].loc[:,'Density'],
                        y = df_oxide[(df_oxide.Formula == 'TiO2') & (df_oxide.E_above_Hull == 0)].loc[:,'IPF'],
                        mode = 'markers', marker=dict(size=18, color='magenta'), text = 'TiO2', name = 'TiO2')

data_IPF = [trace_all,trace_TiC,trace_TiN,trace_TiB,trace_TiO2]
fig_IPF = go.Figure(data_IPF, layout=layout_IPF)
iplot(fig_IPF)

### 3.3.2 Plot material properties from dataframe using 3rd property as color key

In [19]:
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot
layout_IPF_shear_modulus = go.Layout(title= "Ti-based Compounds Bulk Modulus vs. Shear Modulus", hovermode= 'closest',
                   font = dict(family='Times New Roman',size=18),
                   xaxis= dict(title= '$Shear \: Modulus \: [GPa]$',zeroline= False, gridwidth= 2),
                   yaxis= dict(title= '$Bulk \: Modulus [GPa]$',zeroline= False, gridwidth= 2),
                   height = 600,
                   width = 1200,
     showlegend= False              
)

#Scatter plot of collected data. Use df_oxide_all dataframe. df_oxide_all.Density will pull density column. Could change Density or IPF to Molar Volume or simply volume if you wanted.
trace_all_shear = go.Scatter(x = abs(df_oxide.G_VRH), y = df_oxide.K_VRH, mode = 'markers', text = df_oxide.Formula,
                            marker=dict(size=12, color=(df_oxide['poisson_ratio']),
                                colorbar = dict(title={ 'text': "Poisson Ratio", 
                                                     'font': {'family':'Georgia', 'size': 18}} , 
                                              tickfont={'family':'Georgia', 'size': 16 })))#, name = 'All Queries'))

                             
data_IPF_shear = [trace_all_shear]
fig_IPF_shear = go.Figure(data_IPF_shear, layout=layout_IPF_shear_modulus)
iplot(fig_IPF_shear)

Uncomment the cell below to play around with the different parameters we have saved from the query. Below is an example of IPF vs. Elastic Anisotropy.
Quickly uncomment block by highlighting and pressing together Ctrl+'/'

In [20]:
layout_test = go.Layout(title= "Ti compounds", hovermode= 'closest',
                  font = dict(family='Times New Roman',size=36),
                  xaxis= dict(title= 'Density [g/cm<sup>-3</sup>]',zeroline= False, gridwidth= 2), #To change range put command inside xaxis dictionary after gridwidth range=[-XX,XX]
                  yaxis= dict(title= "Young's Modulus [GPa]",zeroline= False, gridwidth= 2,range=[0,600]), #To change range put command inside yaxis dictionary after gridwidth range=[-YY,YY]
                  height = 800,
                  width = 800,
    showlegend= False              
)

#Scatter plot of collected data. Use df_oxide_all dataframe. df_oxide_all.Density will pull density column. Could change Density or IPF to Molar Volume or simply volume if you wanted.
trace_test = go.Scatter(x = df_oxide.Density, y = df_oxide.Y_Modulus, mode = 'markers', text = df_oxide.Formula,
                           marker=dict(size=12, color=(df_oxide['poisson_ratio']),
                               colorbar = dict(title={ 'text': "Poisson Ratio", 
                                                    'font': {'family':'Georgia', 'size': 18}},  
                                             tickfont={'family':'Georgia', 'size': 16 })), )#, name = 'All Queries'))


import plotly.express as px
data_test = [trace_test]
fig_test = go.Figure(data_test, layout=layout_test)
# fig_test.update_layout(color_continuous_scale=px.colors.sequential.Viridis)
iplot(fig_test)